In [ ]:
import os
import openai
import numpy as np
import re
openai.api_key = #here goes your API key
import pandas as pd
import jsonlines
import csv
import random
import json
import time

In [ ]:
feat_dict = {1: 'uncertainty', 2: 'negation', 3: 'time', 4: 'space', 5: 'vocabulary', 6: 'modality', 
             7: 'theory of mind', 8: 'reasoning', 9: 'composition', 10: 'anaphora', 11: 'noise'}
def template(INSTANCIA, NTEMP):
    res= ''
    if NTEMP == 1: #uncertainty
        res = 'You are a helpful expert on linguistics. You must help me annotate the uncertainty level of a given sentence/s. Note that uncertainty refers to epistemic situations involving imperfect or unknown information. I will first give you a few examples to illustrate this (as few-shot learning), and then you will have to determine the level of uncertainty for a new sentence/s on a scale between 0 and 10.\nSentence: "The cat is in the house"\nLevel of uncertainty: 1\nSentence: "She might not do it again"\nLevel of uncertainty: 7\nSentence: "He may come this afternoon":\nLevel of uncertainty: 3\nSentence: "We have no clue about where it is":\nLevel of uncertainty: 8\nSentence: "It is a fact that a square has four sides":\nLevel of uncertainty: 0\nSentence: "It is impossible to know who will win the lottery"\nLevel of uncertainty: 10\nSentence: "' + INSTANCIA + '"\nLevel of uncertainty:'
    elif NTEMP == 2: #negation
        res = 'You are a helpful expert on linguistics. You must help me annotate the negation level of a given sentence/s. Note that negation refers to a denial, contradiction, or negative statement. I will first give you a few examples to illustrate this (as few-shot learning). Then you will have to determine the level of time complexity for a new sentence/s on a scale with levels "no negation" (0), "simple negation" (1), "double negation" (2), "negation with quantification" (3), "very complex negation" (4).\nSentence: \'She has never had a dog\'\nLevel of negation: 1\nSentence: \'It\\\'s untrue that all houses without windows do not have any light\'\nLevel of negation: 4\nSentence: \'I don\\\'t know what I don\\\'t know\'\nLevel of negation: 2\nSentence: \'The suspect is not in the house\'\nLevel of negation: 1\nSentence: \'The car has not been driven by anyone in the team\'\nLevel of negation: 3\nSentence: "'+INSTANCIA+'"\nLevel of negation:'
    elif NTEMP == 3: #time
        res = 'You are a helpful expert on linguistics. You must help me annotate the time complexity level of a given sentence/s. Note that a temporal expression in a text is a sequence of tokens (words, numbers and characters) that denote time, duration, or frequency. I will first give you a few examples to illustrate this (as few-shot learning). Then you will have to determine the level of time complexity for a new sentence/s on a scale with levels "no time expressions" (0), "simple temporal expressions" (1), "double temporal expressions" (2), "complex temporal expressions" (3).\nSentence: "He came before noon"\nLevel of time complexity: 1\nSentence: "The house is blue"\nLevel of time complexity: 0\nSentence: "There\'s a meeting every two weeks"\nLevel of time complexity: 3\nSentence: "The train arrived ten minutes after the plane has left"\nLevel of time complexity: 2\nSentence: "'+ INSTANCIA + '"\nLevel of time complexity:'
    elif NTEMP == 4: #space
        res = 'You are a helpful expert on linguistics. You must help me annotate the space complexity level of a given sentence/s. Note that a spatial expression in a text is a sequence of tokens (words, numbers and characters) relating to the position, area, and size of things. I will first give you a few examples to illustrate this (as few-shot learning). Then you will have to determine the level of space complexity for a new sentence/s on a scale with levels "no space relationships" (0), "simple spatial expressions" (1), "double spatial expressions" (2), "complex spatial expressions" (3).\nSentence: "The pen was on the table"\nLevel of space complexity: 1\nSentence: "There\'s no room between the two cars"\nLevel of space complexity: 2\nSentence: "Tomorrow is a bank holiday"\nLevel of space complexity: 0\nSentence: "The lamp was hanging from two ropes, one attached to the ceiling and the other to the window"\nLevel of space complexity: 5\nSentence: "'+ INSTANCIA + '"\nLevel of space complexity:'
    elif NTEMP == 5: #vocabulary
        res = 'You are a helpful expert on linguistics. You must help me annotate the vocabulary level of a given sentence/s. Note that the vocabulary level is measured by a normalised metric of the log freq of words. I will first give you a few examples to illustrate this (as few-shot learning), and then you will have to determine the vocabulary level for a new sentence/s.\nSentence: “The ball is big”\nLevel of vocabulary: 0.1219\nSentence: “Procrastination jeopardises excellence”\nLevel of vocabulary: 0.4235\nSentence: “The boy must apologise”\nLevel of vocabulary: 0.198\nSentence: “Ignoramus was an ultracrepidarian reposte”\nLevel of vocabulary: 0.8324\nSentence: "' + INSTANCIA + '"\nLevel of vocabulary:'
    elif NTEMP == 6: #modality
        res = 'You are a helpful expert on linguistics. You must help me annotate the level of modality of a given sentence/s. Note that modality refers to a classification of propositions on the basis of whether they claim necessity or possibility or impossibility. I will first give you a few examples to illustrate this (as few-shot learning). Then you will have to determine the level of modality for a new sentence/s on a scale with levels "no modality" (0), "simple modality" (1), "double modality" (2).\nSentence: "The woman walked into a bar"\nLevel of modality: 0\nSentence: "The boy must apologise"\nLevel of modality: 1\nSentence: "The boy thinks we can\'t do it"\nLevel of modality: 3\nSentence: "'+ INSTANCIA + '"\nLevel of modality:'
    elif NTEMP == 7: #theory of mind
        res = 'You are a helpful expert on linguistics. You must help me annotate the level of theory of mind of a given sentence/s. Note that in psychology, theory of mind refers to the capacity to understand other people by ascribing mental states to them. I will first give you a few examples to illustrate this (as few-shot learning). Then you will have to determine the level of theory of mind for a new sentence/s on a scale with levels "no theory of mind" (0), "simple theory of mind" (1), "double theory of mind" (2).\nSentence: “He came to the reception before noon”\nLevel of theory of mind: 0\nSentence: “She didn’t want to buy a car”\nLevel of theory of mind: 1\nSentence: “The boy thinks we can’t do it”\nLevel of theory of mind: 1\nSentence: “The child feared his parents wanted to punish him”\nLevel of theory of mind: 2\nSentence: "' + INSTANCIA + '"\nLevel of theory of mind:'
    elif NTEMP == 8: #reasoning
        res = 'You are a helpful expert on linguistics. You must help me annotate the level of reasoning of a given sentence/s. Note that reasoning is the process of forming conclusions, judgments, or inferences from facts or premises. I will first give you a few examples to illustrate this (as few-shot learning). Then you will have to determine the level of reasoning for a new sentence/s on a scale with levels "no reasoning" (0), "simple reasoning" (1), "complex reasoning" (2).\nSentence: “He tripped because of the step”\nLevel of reasoning: 1\nSentence: “He came before noon with a bag full of presents”\nLevel of reasoning: 0\nSentence: "'+ INSTANCIA + '"\nLevel of reasoning:'
    elif NTEMP == 9: #composition
        res = 'You are a helpful expert on linguistics. You must help me annotate the level of compositionality of a given sentence/s. In semantics, the principle of compositionality states that the meaning of a complex expression is determined by the meanings of its constituent expressions and the rules used to combine them. I will first give you a few examples to illustrate it (as few-shot learning). Then you will have to determine the level of compositionality for a new sentence/s.\nSentence: “He came before noon”\nLevel of compositionality: 0\nSentence: “He came before she arrived”\nLevel of compositionality: 1\nSentence: “The man wearing the tall hat came before she arrived”\nLevel of compositionality: 2\nSentence: “He came before noon with a bag full of presents”\nLevel of compositionality: 0\nSentence: "' + INSTANCIA +'"\nLevel of compositionality:'
    elif NTEMP == 10: #anaphora
        res = 'You are a helpful expert on linguistics. You must help me annotate the level of anaphora of a given sentence/s. Note that anaphora is the use of a linguistic unit, such as a pronoun, to refer to the same person or object as another unit, usually a noun. I will first give you a few examples to illustrate this (as few-shot learning). Then you will have to determine the level of anaphora for a new sentence/s on a scale with levels "no anaphora" (0), "simple (one possible referent)" (1), "complex (>1 possible referents)" (2).\nSentence: “Kim thinks that he is clever”\nLevel of anaphora: 1\nSentence: “While Stuart was telling Susan the news, she laughed at him”\nLevel of anaphora: 2\nSentence: "' + INSTANCIA + '"\nLevel of anaphora:'
    elif NTEMP == 11: #noise
        res = 'You are a helpful expert on linguistics. You must help me annotate the level of noise of a given sentence/s. Note that the level of noise is the number of typos per character with respect to the original text with no typos. I will first give you a few examples to illustrate this (as few-shot learning), and then you will have to determine the level of noise for a new sentence/s.\nSentence: “The ball is big”\nLevel of noise: 0\nSentence: “The bll isbige”\nLevel of noise: 3/13\nSentence: “The boy bust apologise”\nLevel of noise: 1/20\nSentence: "' + INSTANCIA + '"\nLevel of noise:'

    return res

In [ ]:
task = 'epistemic_reasoning'
benchmark = 'bigbench'
df = pd.read_csv(f'./{benchmark}/{task}/{task}_features_sentences.csv')
df

In [ ]:
sentence_dict = {}
for index,row in df.iterrows():
    if row["sentence"] in sentence_dict:
        sentence_dict[row["sentence"]].append(index)
    else:
        sentence_dict[row["sentence"]] = [index]
len(sentence_dict)

In [ ]:
eng = "gpt-4"

with open(f'./{benchmark}/{task}/{eng}_{task}.csv', 'a', newline='', encoding='utf-8') as CSV_file:
    writer_CSV = csv.writer(CSV_file, delimiter=';')
    writer_CSV.writerow(['instance_ID', 'level_ID', 'sentence_ID','sentence','uncertainty','negation','time','space', 
                         'vocabulary','modality', 'theory of mind','reasoning','composition','anaphora','noise'])
    for index,row in df.iterrows():
        print(index)
        
        feat_results = []
        
        first_index = sentence_dict[row["sentence"]][0]
        if first_index < index:
            for feature in feat_dict.keys():
                row[feat_dict[feature]] = df[feat_dict[feature]].iloc[first_index]
                df.iloc[index] = row
                feat_results.append(df[feat_dict[feature]].iloc[first_index])
        else:  
            for feature in feat_dict.keys():
                print('Feature: '+str(feature))
                prompt = template(row["sentence"], feature)
                ok = False
                while not ok:  # to avoid "ServiceUnavailableError: The server is overloaded or not ready yet."
                    try:
                        response = openai.ChatCompletion.create(
                                    model="gpt-4",
                                    messages=[
                                              {"role": "user", "content": prompt}
                                             ],
                                    max_tokens = 10,
                                    temperature = 0
                                    )
                        ok = True

                    except Exception as ex:
                        print("error", ex)
                        print("too much request, sleep for 5 seconds")
                        time.sleep(5)

                response_text = response["choices"][0]["message"]
                content = json.loads(str(response_text))['content']
                row[feat_dict[feature]] = content
                df.iloc[index] = row
                feat_results.append(content)
                
        writer_CSV.writerow([row["instance_ID"],row["level_ID"],row["sentence_ID"],row["sentence"],feat_results[0],
                            feat_results[1],feat_results[2],feat_results[3],feat_results[4],feat_results[5],feat_results[6],
                            feat_results[7],feat_results[8],feat_results[9],feat_results[10]])